In [14]:
from os import environ

from pyspark.sql import SparkSession

environ[
    'PYSPARK_PYTHON'] = "/mnt/ceph/storage/data-in-progress/data-research/web-search/web-archive-query-log/venv/bin/python"
session = SparkSession.builder \
    .master("yarn") \
    .appName("web-archive-query-log-query-length") \
    .config("spark.executor.instances", 3) \
    .getOrCreate()

In [15]:
sc = session.sparkContext
sc

<SparkContext master=yarn appName=web-archive-query-log-query-length>

In [16]:
from pathlib import Path

# TODO: For final evaluation, use the full corpus.
# corpus_dir = Path("/mnt/ceph/storage/data-in-progress/data-research/web-search/web-archive-query-log/focused/corpus")
# queries_dir = corpus_dir / "queries-2023-02-14"
corpus_dir = Path(
    "/mnt/ceph/storage/data-in-progress/data-research/web-search/web-archive-query-log/focused/sample-corpus")
queries_dir = corpus_dir / "queries"

In [17]:
from pathlib import Path

figures_dir = Path("figures")
figures_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
def group(query: dict) -> tuple:
    query_text = query["url_query"]
    return query["service"], query["language"], len(query_text)

In [19]:
from json import loads
from pandas import DataFrame

counts = sc.textFile(f"file://{queries_dir}") \
    .map(lambda line: loads(line)) \
    .filter(lambda query: query["url_query"] is not None) \
    .keyBy(lambda query: query["id"]) \
    .mapValues(group) \
    .countByValue()

In [20]:
df = DataFrame([
    {
        "service": service,
        "language": language,
        "query_length": query_length,
        "count": count,
    }
    for (service, language, query_length), count in counts.items()
])
df

,url_query_length,service,language,count
0,14,baidu,None,17467
1,13,baidu,None,55
2,31,google,en,396
3,41,google,en,181
4,26,google,en,574
...,...,...,...,...
13454,27,youtube,sah,1
13459,82,google,fr,1
13468,115,google,he,1
13476,38,google,ja,1


In [ ]:
top_services = df.groupby("service")["count"].sum().sort_values(ascending=False).head(5).index
top_services

In [ ]:
df_plot = df.copy()
df_plot.loc[
    ~df_plot["service"].isin(top_services),
    "service",
] = "other"

In [ ]:
from seaborn import displot

plot = displot(
    data=df.rename(columns={
        "url_query_length": "Query Length",
        "count": "Count",
        "service": "Service",
    }),
    x="Query Length",
    weights="Count",
    hue="Service",
    binwidth=1,
    multiple="stack",
    aspect=2,
    linewidth=0,
)
plot.set_axis_labels("Query Length", "Count")
# plot.savefig(figures_dir / "query-length-histogram.pdf")
# plot.savefig(figures_dir / "query-length-histogram.png")
plot

/opt/conda/lib/python3.8/site-packages/seaborn/distributions.py:269: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  baselines.iloc[:, cols] = (curves
